In [18]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import sqlalchemy
import os

In [2]:
conn = psycopg2.connect(
    user="dcat", 
    password="dcat", 
    host="127.0.0.1", 
    port="54320", 
    database="dcat")

In [21]:
# vault login -address="https -method=oidc
# vault read --address="https://vault.adeo.no" postgresql/preprod-fss/creds/datacatalog-term-admin

conn = psycopg2.connect(
    user="v-oidc-Pau-data-cat-JttunzZohPHCVqCALH7e-1574349384", 
    password="A1a-KVbk7te3wn4ndtue", 
    host="A01DBVL010.adeo.no", 
    port="5432", 
    database="data-catalog-metadata")

In [22]:
cur = conn.cursor()

In [23]:
print ( conn.get_dsn_parameters(),"\n")

{'user': 'v-oidc-Pau-data-cat-JttunzZohPHCVqCALH7e-1574349384', 'dbname': 'data-catalog-metadata', 'host': 'A01DBVL010.adeo.no', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



In [24]:
cur.execute("SELECT version();")
record = cur.fetchone()
print("You are connected to - ", record,"\n")

You are connected to -  ('PostgreSQL 11.3 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-36), 64-bit',) 



In [25]:
cur.execute("CREATE EXTENSION IF NOT EXISTS pgcrypto;")

InsufficientPrivilege: permission denied to create extension "pgcrypto"
HINT:  Must be superuser to create this extension.


In [33]:
conn.commit()

In [27]:
cur.execute("drop index if exists idx_tbl_edge_n1")
cur.execute("drop index if exists idx_tbl_edge_n2")
cur.execute("drop table if exists tbl_edge")

# Node table

In [31]:
cur.execute("drop table if exists tbl_node")
cur.execute("""
create table tbl_node 
(  
  id SERIAL PRIMARY KEY,
  guid UUID default gen_random_uuid(),
  prop JSONB, 
  created TIMESTAMP default now() 
);  
""")

SyntaxError: syntax error at or near "//"
LINE 5:   //guid UUID default gen_random_uuid(),
          ^


In [34]:
cur.execute("drop table if exists tbl_node")
cur.execute("""
create table tbl_node 
(  
  id SERIAL PRIMARY KEY,
  prop JSONB, 
  created TIMESTAMP default now() 
);  
""")

In [35]:
sql = "select * from tbl_node"
df = sqlio.read_sql_query(sql, conn)
df.head()

,id,prop,created


In [46]:
conn.commit()
cur.execute('grant all privileges on table tbl_node to "data-catalog-graph-admin"')

UndefinedObject: role "data-catalog-graph-admin" does not exist


In [48]:
conn.commit()
sql = 'select * from pg_catalog.pg_user'
df_users = sqlio.read_sql_query(sql, conn)

,usename,usesysid,usecreatedb,usesuper,userepl,usebypassrls,passwd,valuntil,useconfig
0,v-kubernet-spleis-u-FdEAaQlqdGCatfoIRtE3-15743...,17371107,False,False,False,False,********,2019-11-21 17:26:25+01,None
1,v-kubernet-spock-us-U6RQVStzgsgqkXI0m3BV-15743...,17370397,False,False,False,False,********,2019-11-21 16:47:52+01,None
2,postgres,10,True,True,True,True,********,None,None
3,v-oidc-Pau-data-cat-JttunzZohPHCVqCALH7e-15743...,17370925,False,False,False,False,********,2019-11-21 17:16:29+01,None
4,v-kubernet-spleis-a-E4MMnnx7Uz26QPWXM3cT-15743...,17371106,False,False,False,False,********,2019-11-21 17:26:24+01,None


# Edge description/attributes

In [36]:
cur.execute("drop table if exists tbl_edge_desc")
cur.execute("""
create table tbl_edge_desc  -- Relationship description  
(  
  id SERIAL PRIMARY KEY, 
  prop TEXT 
);
""")

In [37]:
sql = "select * from tbl_edge_desc"
df = sqlio.read_sql_query(sql, conn)
df.head()

,id,prop


# Edges 

In [13]:
cur.execute("drop table if exists tbl_edge")
cur.execute("""
create table tbl_edge     -- What is relationship between ID1 to ID2      
(  
  n1 int references tbl_node(id),    
  n2 int references tbl_node(id),
  guid UUID default gen_random_uuid(),
  prop int2[],
  created timestamp default now(),  
  check (n1<>n2),  
  unique (n1,n2)  
);  
""")

In [38]:
cur.execute("drop table if exists tbl_edge")
cur.execute("""
create table tbl_edge     -- What is relationship between ID1 to ID2      
(  
  n1 int references tbl_node(id),    
  n2 int references tbl_node(id),
  prop int2[],
  created timestamp default now(),  
  check (n1<>n2),  
  unique (n1,n2)  
);  
""")

In [39]:
sql = "select * from tbl_edge"
df = sqlio.read_sql_query(sql, conn)
df.head()

,n1,n2,prop,created


In [40]:
cur.execute("drop index if exists idx_tbl_edge_n1")
cur.execute("create index idx_tbl_edge_n1 on tbl_edge(n1)")

In [41]:
cur.execute("drop index if exists idx_tbl_edge_n2")
cur.execute("create index idx_tbl_edge_n2 on tbl_edge(n2)")

In [42]:
conn.commit()